In [ ]:
import pandas as pd
import os
import shutil

In [ ]:
def extract_split_info(purpose_text):
    try:
        words = purpose_text.split()
        from_value = int(words[7].replace("/-", ""))
        to_value = int(words[12].replace("/-", ""))
        ratio = f"{to_value}:{from_value}"
        divisor = from_value / to_value
        return ratio, divisor
    except (IndexError, ValueError):
        return None, None
    
split_df = pd.read_csv('SPLIT.csv')
split_df[['RATIO', 'DIVISOR']] = split_df['PURPOSE'].apply(lambda x: pd.Series(extract_split_info(x)))
split_df.to_csv("Splitx.csv", index=False)

In [ ]:
def extract_bonus_info(purpose_text):
    try:
        words = purpose_text.split()
        ratio_parts = words[1].split(":")
        shares_before_bonus = int(ratio_parts[1])
        bonus_shares = int(ratio_parts[0])
        ratio = f"{bonus_shares}:{shares_before_bonus}"
        divisor = (bonus_shares + shares_before_bonus)/shares_before_bonus
        return ratio, divisor
    except (IndexError, ValueError):
        return None, None

bonus_df = pd.read_csv('BONUS.csv')
bonus_df[['RATIO', 'DIVISOR']] = bonus_df['PURPOSE'].apply(lambda x: pd.Series(extract_bonus_info(x)))
bonus_df.to_csv('BONUSx.csv', index=False)

In [ ]:
def copy_files(csv_file, source_dir, dest_dir):
    
    symbols = pd.read_csv(csv_file)['SYMBOL'].unique()
    os.makedirs(dest_dir, exist_ok=True)
    copied_files = 0
    for symbol in symbols:
        source_file = os.path.join(source_dir, f"{symbol}.csv")
        dest_file = os.path.join(dest_dir, f"{symbol}.csv")
        if os.path.exists(source_file):
            shutil.copy2(source_file, dest_file)
            copied_files += 1
            print(f"Copied: {symbol}.csv")
        else:
            print(f"File not found: {symbol}.csv")
    print(f"\nTotal files copied: {copied_files}")

copy_files('BONUSx.csv', "../history/history", "/copiedfiledirectorypath")

In [ ]:
def adjust_file(symbol, ex_date, divisor, file_path, output_dir):
    try:
        df = pd.read_csv(file_path, parse_dates=['date'])
        ex_date = pd.to_datetime(ex_date, format='%d-%b-%y')

        if pd.api.types.is_datetime64tz_dtype(df['date']):
            ex_date = ex_date.tz_localize(df['date'].dt.tz)

        condition = df['date'] < ex_date
        df.loc[condition, ['open', 'high', 'low', 'close']] /= divisor
        
        df[['open', 'high', 'low', 'close']] = df[['open', 'high', 'low', 'close']].round(2)
        
        os.makedirs(output_dir, exist_ok=True)
        adjusted_file_path = os.path.join(output_dir, f"{symbol}_adjusted.csv")
    
        df.to_csv(adjusted_file_path, index=False)
        print(f"File for {symbol} adjusted and saved at {adjusted_file_path}")
        
    except Exception as e:
        print(f"Error adjusting file for {symbol}: {e}")

In [ ]:
source_dir = "/copiedfiledirectorypath"
output_dir = "/adjustedfilepath"

In [ ]:
for index, row in bonus_df.iterrows():
    symbol = row['SYMBOL']
    ex_date = row['EX-DATE']
    divisor = row['DIVISOR']
    file_path = os.path.join(source_dir, f"{symbol}.csv")
    
    if os.path.exists(file_path):
        adjust_file(symbol, ex_date, divisor, file_path, output_dir)
    else:
        print(f"File for {symbol} not found in source directory.")

In [ ]:
for index, row in split_df.iterrows():
    symbol = row['SYMBOL']
    ex_date = row['EX-DATE']
    divisor = row['DIVISOR']
    file_path = os.path.join(source_dir, f"{symbol}.csv")
    
    if os.path.exists(file_path):
        adjust_file(symbol, ex_date, divisor, file_path, output_dir)
    else:
        print(f"File for {symbol} not found in source directory.")